In [39]:
import pandas as pd

import DataRetriever as dr
import SARIMAX_forecast
import FFT_forecast

RETRIEVER = dr.DataRetriever()
CON_ATTRIBUTES = RETRIEVER.get_attributes(file_name='consuming_attributes.pkl')
PV_ATTRIBUTES = RETRIEVER.get_attributes(file_name='producing_attributes.pkl')
FLEX_ATTRIBUTES = ["Load_ClothesWasherPowerWithStandby", "Elec_PowerDishwasher", "Load_DryerPowerTotal"]
FIXED_ATTRIBUTES = list(set(CON_ATTRIBUTES) - set(FLEX_ATTRIBUTES))

In [40]:
def rescheduler(year, month, day):
    date = pd.Timestamp(year=year, month=month, day=day)
    pv = RETRIEVER.get_data(file_name='All-Subsystems-hour-Year2.pkl')[PV_ATTRIBUTES].sum(axis=1).clip(lower=0) / 1000
    fixed = RETRIEVER.get_data(file_name='All-Subsystems-hour-Year2.pkl')[FIXED_ATTRIBUTES].sum(axis=1).clip(lower=0) / 1000

    flex_clothes = RETRIEVER.get_data(file_name='All-Subsystems-hour-Year2.pkl')["Load_ClothesWasherPowerWithStandby"].clip(lower=0) / 1000
    flex_dish = RETRIEVER.get_data(file_name='All-Subsystems-hour-Year2.pkl')["Elec_PowerDishwasher"].clip(lower=0) / 1000
    flex_dryer = RETRIEVER.get_data(file_name='All-Subsystems-hour-Year2.pkl')["Load_DryerPowerTotal"].clip(lower=0) / 1000

    # 7 days of training data - subject to be changed, depends on fcast model
    # pv = pv.loc[(pv.index >= date - pd.Timedelta(days=7)) & (pv.index < date)]
    fixed = fixed.loc[(fixed.index >= date - pd.Timedelta(days=7)) & (fixed.index < date)]

    flex_clothes = flex_clothes.loc[(flex_clothes.index >= date) & (flex_clothes.index < date + pd.Timedelta(days=3))]
    flex_dish = flex_dish.loc[(flex_dish.index >= date) & (flex_dish.index < date + pd.Timedelta(days=3))]
    flex_dryer = flex_dryer.loc[(flex_dryer.index >= date) & (flex_dryer.index < date + pd.Timedelta(days=3))]

    # ------------------ Forecast PV ------------------
    pv_pred = SARIMAX_forecast.sarimax_prediction(date, pv)

    # ------------------ Forecast Fixed ------------------
    fixed_pred = pd.DataFrame(FFT_forecast.fourierExtrapolation(data=fixed, number_of_predictions=72, n_sinusoids=70)[len(fixed):], index=pv_pred.index[:72])
    #pd.DataFrame(fixed[:len(flex_clothes)]).set_index(flex_dish.index)


    # Concat results
    df = pd.concat([pv_pred.reindex(flex_dish.index), fixed_pred.reindex(flex_dish.index), flex_clothes, flex_dish, flex_dryer], axis=1)
    # Easy way to remove column names
    df = df.T.reset_index(drop=True).T
    df.rename(columns={0: "PV", 1: "FIXED", 2: "FLEX_CLOTHES", 3: "FLEX_DISH", 4: "FLEX_DRYER"}, inplace=True)

    # Stand-by values set to 0, simplifies the code
    df.loc[df["FLEX_CLOTHES"] <= 0.002947, "FLEX_CLOTHES"] = 0

    df["Balance"] = df["PV"] - (df["FIXED"] + df["FLEX_CLOTHES"] + df["FLEX_DISH"] + df["FLEX_DRYER"])

    # pd.DataFrames with flexible appliances turned on and surplus where at least one flexible is not turned on.
    deficit = df[(df["Balance"] < 0) & ((df["FLEX_DISH"] > 0) | (df["FLEX_DRYER"] > 0) | (df["FLEX_CLOTHES"] > 0))]
    surplus = df[(df["Balance"] > 0) & ((df["FLEX_DISH"] == 0) | (df["FLEX_DRYER"] == 0) | (df["FLEX_CLOTHES"] == 0))]

    columns_dict = {0: "FLEX_CLOTHES", 1: "FLEX_DISH", 2: "FLEX_DRYER"}
    res_df = pd.DataFrame(columns=["Planned Time", "Appliance", "Suggested Time"])

    # Loops over all rows in the deficit to find better times to turn on appliances.
    for idx, row in deficit.iterrows():
        while row[["FLEX_CLOTHES", "FLEX_DISH", "FLEX_DRYER"]].max() != 0: # Thus, a flexible appliance is active.
            col = columns_dict[row[["FLEX_CLOTHES", "FLEX_DISH", "FLEX_DRYER"]].argmax()] # Column of the largest deficit.

            deficit_value = deficit.at[idx, col] # Significance of deficit
            surplus_possible = surplus[(surplus["Balance"] >= deficit_value) & (surplus[col] == 0)] # Possible time slots

            suggested_time = surplus_possible.index[surplus_possible.index.get_indexer([idx], method='nearest')][0] # Closest time with enough surplus

            surplus.at[suggested_time, col] = deficit_value
            surplus.at[suggested_time, "Balance"] -= deficit_value # Change balance in the surplus df

            deficit.at[idx, col] = 0 # Set the deficit value to 0

            res_df.loc[len(res_df)] = (idx, col, suggested_time) # Insert rows to final result

    return res_df

In [45]:
suggested_rescheduler = rescheduler(2016, 1, 21)
suggested_rescheduler

,Planned Time,Appliance,Suggested Time
0,2016-01-22 20:00:00,FLEX_DISH,2016-01-22 12:00:00
1,2016-01-22 21:00:00,FLEX_DISH,2016-01-22 11:00:00
2,2016-01-22 22:00:00,FLEX_DISH,2016-01-22 10:00:00
3,2016-01-23 11:00:00,FLEX_CLOTHES,2016-01-22 11:00:00
4,2016-01-23 12:00:00,FLEX_DRYER,2016-01-21 15:00:00
5,2016-01-23 14:00:00,FLEX_CLOTHES,2016-01-22 12:00:00
6,2016-01-23 15:00:00,FLEX_DRYER,2016-01-21 14:00:00
7,2016-01-23 15:00:00,FLEX_CLOTHES,2016-01-22 10:00:00
8,2016-01-23 21:00:00,FLEX_DISH,2016-01-22 09:00:00
9,2016-01-23 22:00:00,FLEX_DISH,2016-01-21 16:00:00


In [42]:
date = pd.Timestamp(year=2016, month=1, day=21)
pv = RETRIEVER.get_data(file_name='All-Subsystems-hour-Year2.pkl')[PV_ATTRIBUTES].sum(axis=1).clip(lower=0) / 1000
fixed = RETRIEVER.get_data(file_name='All-Subsystems-hour-Year2.pkl')[FIXED_ATTRIBUTES].sum(axis=1).clip(lower=0) / 1000

flex_clothes = RETRIEVER.get_data(file_name='All-Subsystems-hour-Year2.pkl')["Load_ClothesWasherPowerWithStandby"].clip(lower=0) / 1000
flex_dish = RETRIEVER.get_data(file_name='All-Subsystems-hour-Year2.pkl')["Elec_PowerDishwasher"].clip(lower=0) / 1000
flex_dryer = RETRIEVER.get_data(file_name='All-Subsystems-hour-Year2.pkl')["Load_DryerPowerTotal"].clip(lower=0) / 1000

pv = pv.loc[(pv.index >= date) & (pv.index < date + pd.Timedelta(days=3))]
fixed = fixed.loc[(fixed.index >= date) & (fixed.index < date + pd.Timedelta(days=3))]

flex_clothes = flex_clothes.loc[(flex_clothes.index >= date) & (flex_clothes.index < date + pd.Timedelta(days=3))]
flex_dish = flex_dish.loc[(flex_dish.index >= date) & (flex_dish.index < date + pd.Timedelta(days=3))]
flex_dryer = flex_dryer.loc[(flex_dryer.index >= date) & (flex_dryer.index < date + pd.Timedelta(days=3))]


df = pd.concat([pv.reindex(flex_dish.index), fixed.reindex(flex_dish.index), flex_clothes, flex_dish, flex_dryer], axis=1)
# Easy way to remove column names
df = df.T.reset_index(drop=True).T
df.rename(columns={0: "PV", 1: "FIXED", 2: "FLEX_CLOTHES", 3: "FLEX_DISH", 4: "FLEX_DRYER"}, inplace=True)

# Stand-by values set to 0, simplifies the code
df.loc[df["FLEX_CLOTHES"] <= 0.002947, "FLEX_CLOTHES"] = 0

df["Balance"] = df["PV"] - (df["FIXED"] + df["FLEX_CLOTHES"] + df["FLEX_DISH"] + df["FLEX_DRYER"])
df

,PV,FIXED,FLEX_CLOTHES,FLEX_DISH,FLEX_DRYER,Balance
Timestamp,,,,,,
2016-01-21 00:00:00,0.018528,1.907302,0.0,0.000000,0.0,-1.888774
2016-01-21 01:00:00,0.018481,1.625556,0.0,0.000000,0.0,-1.607075
2016-01-21 02:00:00,0.018412,1.846838,0.0,0.000000,0.0,-1.828426
2016-01-21 03:00:00,0.018334,1.778191,0.0,0.000000,0.0,-1.759857
2016-01-21 04:00:00,0.018563,1.591810,0.0,0.000000,0.0,-1.573246
...,...,...,...,...,...,...
2016-01-23 19:00:00,0.018728,4.338868,0.0,0.000000,0.0,-4.320140
2016-01-23 20:00:00,0.018731,4.066591,0.0,0.000000,0.0,-4.047859
2016-01-23 21:00:00,0.018751,2.676231,0.0,0.357464,0.0,-3.014945


In [46]:
balance_pre = sum(df["Balance"][df["Balance"] < 0])
balance_pre

-168.17352059371296

In [47]:
for idx, row in suggested_rescheduler.iterrows():
    df.at[row["Suggested Time"], row["Appliance"]] = df.at[row["Planned Time"], row["Appliance"]]
    df.at[row["Planned Time"], row["Appliance"]] = 0

In [48]:
df["New_Balance"] = df["PV"] - (df["FIXED"] + df["FLEX_CLOTHES"] + df["FLEX_DISH"] + df["FLEX_DRYER"])
df

,PV,FIXED,FLEX_CLOTHES,FLEX_DISH,FLEX_DRYER,Balance,New_Balance
Timestamp,,,,,,,
2016-01-21 00:00:00,0.018528,1.907302,0.0,0.0,0.0,-1.888774,-1.888774
2016-01-21 01:00:00,0.018481,1.625556,0.0,0.0,0.0,-1.607075,-1.607075
2016-01-21 02:00:00,0.018412,1.846838,0.0,0.0,0.0,-1.828426,-1.828426
2016-01-21 03:00:00,0.018334,1.778191,0.0,0.0,0.0,-1.759857,-1.759857
2016-01-21 04:00:00,0.018563,1.591810,0.0,0.0,0.0,-1.573246,-1.573246
...,...,...,...,...,...,...,...
2016-01-23 19:00:00,0.018728,4.338868,0.0,0.0,0.0,-4.320140,-4.320140
2016-01-23 20:00:00,0.018731,4.066591,0.0,0.0,0.0,-4.047859,-4.047859
2016-01-23 21:00:00,0.018751,2.676231,0.0,0.0,0.0,-3.014945,-2.657480
